# gradescope

> This module process the gradescope generated csv grades file, and use canvas api to post grade to students, with late penalty.

In [ ]:
#| default_exp gradescope

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import canvasapi
from canvasapi import Canvas
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
#| export
#| hide
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
#| export
class gradescope_grade:
    def __init__(self,
                 credentials_fp = "", # credential file path. [Template of the credentials.json](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/nbs/credentials.json)
                 API_URL="https://canvas.ucsd.edu", # the domain name of canvas
                 course_id="", # Course ID, can be found in the course url
                 assignment_id=-1, # assignment id, can be found in the canvas assignment url
                 gradescope_fp="", # gradescope csv file path 
                 verbosity=0 # Controls the verbosity: 0 = Silent, 1 = print all messages
                ):
        "Initialize Canvas Group within a Group Set and its appropriate memberships"
        self.API_URL = API_URL
        self.canvas = None
        self.course = None
        self.users = None
        self.email_to_canvas_id = None
        self.canvas_id_to_email = None
        self.API_KEY = None
        self.verbosity = verbosity
        self.assignment = None
        self.gradescope = None
        
        # initialize by the input parameter
        if credentials_fp != "":
            self.auth_canvas(credentials_fp)
        if course_id != "":
            self.set_course(course_id)
        if assignment_id != -1:
            self.link_assignment(assignment_id)
        if gradescope_fp != "":
            self.load_gradescope_csv(gradescope_fp)
        
    def auth_canvas(self,
                    credentials_fp: str # the Authenticator key generated from canvas
                   ):
        "Authorize the canvas module with API_KEY"
        with open(credentials_fp, "r") as f:
            credentials = json.load(f)
        self.API_KEY = credentials["Canvas Token"]
        self.GITHUB_TOKEN = credentials["GitHub Token"]
        self.canvas = Canvas(self.API_URL, self.API_KEY)
        # test authorization
        _ = self.canvas.get_activity_stream_summary()
        if self.verbosity != 0:
            print(f"{bcolors.OKGREEN}Authorization Successful!{bcolors.ENDC}")
        
    def set_course(self, 
                   course_id: int # the course id of the target course
                  ):
        "Set the target course by the course ID"
        self.course = self.canvas.get_course(course_id)
        if self.verbosity != 0:
            print(f"Course Set: {bcolors.OKGREEN} {self.course.name} {bcolors.ENDC}")
            print(f"Getting List of Users... This might take a while...")
        self.users = list(self.course.get_users())
        if self.verbosity != 0:
            print(f"Users Fetch Complete! The course has {bcolors.OKBLUE}{len(self.users)}{bcolors.ENDC} users.")
        self.email_to_canvas_id = {}
        self.canvas_id_to_email = {}
        for u in self.users:
            try:
                self.email_to_canvas_id[u.email.split("@")[0]] = u.id
                self.canvas_id_to_email[u.id] = u.email.split("@")[0]
            except Exception:
                if self.verbosity != 0:
                    print(f"{bcolors.WARNING}Failed to Parse email and id"
                          f" for {bcolors.UNDERLINE}{u.short_name}{bcolors.ENDC}{bcolors.ENDC}")

    def link_assignment(self,
                        assignment_id: int # assignment id, found at the url of assignmnet tab
                       ) -> canvasapi.assignment.Assignment: # target assignment
        "Link the target assignment on canvas"
        assignment = self.course.get_assignment(assignment_id)
        if self.verbosity != 0:
            print(f"Assignment {bcolors.OKGREEN+assignment.name+bcolors.ENDC} Link!")
        self.assignment = assignment
        return assignment
                    
    def load_gradescope_csv(self,
                            csv_pf:str # csv file path 
                           ):
        "Load gradescope exported csv file"
        self.gradescope = pd.read_csv(csv_pf)
        self.gradescope['Email'] = self.gradescope["Email"].str.split("@").str[0]
        self.gradescope = self.gradescope.set_index("Email")
        self.gradescope = self.gradescope.fillna(0)
        
    def calculate_late_hour(self,
                            target_assignment:str, # target assignment name. Must in the column of gradescope csv 
                           ) -> pd.Series: # late hours of the target assignment
        "Calculate the late hours of each submission of the target assignment"
        late_col_name = f"{target_assignment} - Lateness (H:M:S)"
        late_col = self.gradescope[late_col_name]
        # calculate how many slip day (hours) used for this assignment.
        late_hours = (
            late_col.str.split(":").str[0].astype(int) + 
            np.ceil(late_col.str.split(":").str[1].astype(int)/60)
        )
        return late_hours
        
    def calculate_credit_balance(self,
                                 passed_assignments:[str], # list of passed assignment. Must in the column of gradescope csv
                                 total_credit = 120 # total number of allowed late hours
                                ) -> dict: # {email: credit balance} late credit balance of each students
        "Calculate the balance of late hours from the gradescope file"
        self.gradescope["late balance"] = total_credit
        for passed in passed_assignments:
            late_hours = self.calculate_late_hour(passed)
            self.gradescope["late balance"] = self.gradescope["late balance"] - late_hours
        return self.gradescope["late balance"]
    
    def calculate_total_score(self,
                              components:[str], # components of a single assignment. Must in the column of gradescope csv
                             ) -> pd.Series:
        "Calculate the total score of an assignment"
        self.gradescope["target_total"] = 0
        for component in components:
            self.gradescope["target_total"] += self.gradescope[component]
        return self.gradescope["target_total"]
    
    def _post_grade(self,
                   student_id: int, # canvas student id of student. found in self.email_to_canvas_id
                   grade: float, # grade of that assignment
                   text_comment="", # text comment of the submission. Can feed
                  ) -> canvasapi.submission.Submission: # created submission
        "Post grade and comment to canvas to the target assignment"
        submission = self.assignment.get_submission(student_id)
        edited = submission.edit(
            submission={
                'posted_grade': grade
            }, comment={
                'text_comment': text_comment
            }
        )
        if self.verbosity != 0:
            print(f"Grade for {bcolors.OKGREEN+email+bcolors.ENDC} Posted!")
        return edited
    
    def post_to_canvas(self,
                       target_assignment:str, # target assignment name to grab the late time. Must in the column of gradescope csv 
                       passed_assignments:[str], # list of passed assignment. Must in the column of gradescope csv
                       components:[str], # components of a single assignment. Must in the column of gradescope csv
                       post=True, # for testing purposes. Can hault the post operation
                      ):
        "Post grade to canvas with late penalty."
        if self.gradescope is None:
            raise ValueError("Gradescope CSV has not been loaded. Please set it via process_grade.load_gradescope_csv")
        credit_balance = self.calculate_credit_balance(passed_assignments)
        late_hours = self.calculate_late_hour(target_assignment)
        if len(components) > 1:
            total_score = self.calculate_total_score(components)
        else:
            total_score = self.gradescope[target_assignment]
        # Post Grade
        for email, _ in self.gradescope.iterrows():
            remaining = credit_balance[email]
            score, slip_hour = round(total_score[email], 4), late_hours[email]
            message = f"{target_assignment}: \n"
            if slip_hour > 0:
                # means late submission. Check remaining slip day
                message += f"Late Submission: {int(slip_hour)} Hours Late\n"
                if remaining - slip_hour < 0:
                    message += "Insufficient Slip Credit. 25% late penalty applied\n"
                    score = round(score * 0.75, 4)
                else:
                    message += "Slip Credit Used. No late penalty applied\n"
            else:
                message += "Submitted intime\n"
            balance_after = remaining - slip_hour
            message += f"Remaining Slip Credit: {int(balance_after)} Hours"
            try:
                if post:
                    student_id = self.email_to_canvas_id[email.split("@")[0]]
                    self._post_grade(grade=score, student_id=student_id, text_comment=message)
                else:
                    print(f"{bcolors.WARNING}Post Disabled{bcolors.ENDC}\n"
                          f"The message is: \n{bcolors.OKGREEN+message+bcolors.ENDC}"
                         )
            except Exception as e:
                print(f"Studnet: {bcolors.WARNING+email+bcolors.ENDC} Not found on canvas. \n"
                      f"Maybe Testing Account or Dropped Student")
                print(e)
                pass
        

In [ ]:
show_doc(gradescope_grade)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L27){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade

>      gradescope_grade (credentials_fp='', API_URL='https://canvas.ucsd.edu',
>                        course_id='', assignment_id=-1, gradescope_fp='',
>                        verbosity=0)

Initialize Canvas Group within a Group Set and its appropriate memberships

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| credentials_fp | str |  | credential file path. [Template of the credentials.json](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/nbs/credentials.json) |
| API_URL | str | https://canvas.ucsd.edu | the domain name of canvas |
| course_id | str |  | Course ID, can be found in the course url |
| assignment_id | int | -1 | assignment id, can be found in the canvas assignment url |
| gradescope_fp | str |  | gradescope csv file path |
| verbosity | int | 0 | Controls the verbosity: 0 = Silent, 1 = print all messages |

In [ ]:
show_doc(gradescope_grade.auth_canvas)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L58){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.auth_canvas

>      gradescope_grade.auth_canvas (credentials_fp:str)

Authorize the canvas module with API_KEY

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| credentials_fp | str | the Authenticator key generated from canvas |

In [ ]:
show_doc(gradescope_grade.set_course)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L72){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.set_course

>      gradescope_grade.set_course (course_id:int)

Set the target course by the course ID

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| course_id | int | the course id of the target course |

In [ ]:
show_doc(gradescope_grade.link_assignment)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L94){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.link_assignment

>      gradescope_grade.link_assignment (assignment_id:int)

Link the target assignment on canvas

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| assignment_id | int | assignment id, found at the url of assignmnet tab |
| **Returns** | **Assignment** | **target assignment** |

In [ ]:
show_doc(gradescope_grade.load_gradescope_csv)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L104){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.load_gradescope_csv

>      gradescope_grade.load_gradescope_csv (csv_pf:str)

Load gradescope exported csv file

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| csv_pf | str | csv file path |

In [ ]:
show_doc(gradescope_grade.calculate_late_hour)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L113){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.calculate_late_hour

>      gradescope_grade.calculate_late_hour (target_assignment:str)

Calculate the late hours of each submission of the target assignment

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| target_assignment | str | target assignment name. Must in the column of gradescope csv |
| **Returns** | **Series** | **late hours of the target assignment** |

In [ ]:
show_doc(gradescope_grade.calculate_credit_balance)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L126){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.calculate_credit_balance

>      gradescope_grade.calculate_credit_balance
>                                                 (passed_assignments:[<class'st
>                                                 r'>], total_credit=120)

Calculate the balance of late hours from the gradescope file

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| passed_assignments | [<class 'str'>] |  | list of passed assignment. Must in the column of gradescope csv |
| total_credit | int | 120 | total number of allowed late hours |
| **Returns** | **dict** |  | **{email: credit balance} late credit balance of each students** |

In [ ]:
show_doc(gradescope_grade.calculate_total_score)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L137){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.calculate_total_score

>      gradescope_grade.calculate_total_score (components:[<class'str'>])

Calculate the total score of an assignment

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| components | [<class 'str'>] | components of a single assignment. Must in the column of gradescope csv |
| **Returns** | **Series** |  |

In [ ]:
show_doc(gradescope_grade._post_grade)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L146){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade._post_grade

>      gradescope_grade._post_grade (student_id:int, grade:float,
>                                    text_comment='')

Post grade and comment to canvas to the target assignment

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| student_id | int |  | canvas student id of student. found in self.email_to_canvas_id |
| grade | float |  | grade of that assignment |
| text_comment | str |  | text comment of the submission. Can feed |
| **Returns** | **Submission** |  | **created submission** |

In [ ]:
show_doc(gradescope_grade.post_to_canvas)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/gradescope.py#L164){target="_blank" style="float:right; font-size:smaller"}

### gradescope_grade.post_to_canvas

>      gradescope_grade.post_to_canvas (target_assignment:str,
>                                       passed_assignments:[<class'str'>],
>                                       components:[<class'str'>], post=True)

Post grade to canvas with late penalty.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_assignment | str |  | target assignment name to grab the late time. Must in the column of gradescope csv |
| passed_assignments | [<class 'str'>] |  | list of passed assignment. Must in the column of gradescope csv |
| components | [<class 'str'>] |  | components of a single assignment. Must in the column of gradescope csv |
| post | bool | True | for testing purposes. Can hault the post operation |

In [ ]:
#| hide
# grade.calculate_credit_balance(["Assignment 1"])

In [ ]:
#| hide
# grade.calculate_total_score(["Assignment 1", "Assignment 1 - Manual Grading"])

In [ ]:
#| hide
# grade.calculate_late_hour("Assignment 1")

In [ ]:
#| hide
# # This is forbidden... Ask for the privilage?
# assignment.submit(
#     submission={
#         "submission_type": "oneline_upload",
#         "user_id": student_id,
#         "submitted_at": submission_time
#     }
# )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()